# OKM Van Loon #

This script reads in an Excel file containing a Bill of Material (BOM), and an Excel file containing additional information such as prices per ingredient, waste per HF, packaging, etc. It then combines these two to calculate the cost of the products. It then outputs these costs as an Excel file.

## Set-up ##

### Imports ###

In [344]:
import pandas as pd
import numpy as np

### Functions ###

In [345]:
def calc_cost(bom : pd.DataFrame, costs : pd.DataFrame, exclude : list, q : str, p : str) -> float:
    """ 
    calculate cost for an object given its BOM 
    
    args
    ----
    bom : DataFrame containing the BOM
    costs : DataFrame containing the costs
    exclude : list of BOM items to exclude
    q : name of the bom DataFrame column containing the quantity
    p : name of the costs DataFrame column containing the price

    returns
    -------
    cost : the cost of the object
    """

    cost = 0.0

    for index, row1 in bom.iterrows():
        item = row1['hf_nr']

        # filter items
        if not item in exclude:
            for index, row2 in costs.iterrows():
                if item == str(row2["INGREDIENT CODE"]):
                    cost += float(row2[p]) * float(row1[q])
    
    return cost

### Objects ###

In [346]:
class recipe:
    """ a recipe """

    def __init__(self, name : str, id : str, data : pd.DataFrame, cost: float=0.0, HFs: list=[]) -> None:
        """ initialise an instance of recipe"""
        self.name = name
        self.id = id
        self.data = data
        self.cost = cost
        self.HFs = HFs

    def __str__(self) -> str:
        """ set the string representation of a recipe """
        return f'{self.id} {self.name}'
    
    def calc_recipe_cost(self) -> float:
        """ invoke the calc_cost function with the recipe parameters"""
        return calc_cost(bom=self.data, costs=cost_weight_data, exclude=np.array(list(product_data_HF) + list(product_data_packaging)), q='Aantal (Basis)', p='PRICE Q1')
    
class HF:
    """ an HF (halffabricaat)"""

    def __init__(self, name: str, id: str, cost: float, weight: float, waste: float, data: pd.DataFrame) -> None:
        """ initialise an instance of HF"""
        self.name = name
        self.id = id
        self.cost = cost
        self.weight = weight
        self.waste = waste
        self.data = data
    
    def __str__(self):
        """ set the string representation of an HF"""
        return f'{self.id} {self.name}'
    
    def calc_hf_cost(self) -> float:
        """ invoke the calc_cost function with the HF parameters"""
        return calc_cost(bom=self.data, costs=cost_weight_data, exclude=np.array(list(product_data_HF) + list(product_data_packaging)), q='Aantal (Basis)', p='PRICE Q1')

## Data preparation ##

### Data loading ###

#### BOM ####


In [347]:
bom_data_raw = pd.read_excel("recepten.xlsx", skiprows=1, header=None, decimal=",")

#### Costs & weights ####

In [348]:
cost_weight_data = pd.read_excel("Input Price List + Grammage.xlsx", sheet_name="PriceList", header=0).astype({'INGREDIENT CODE': 'string'})

#### Waste ####

In [349]:
waste_data = pd.read_excel("Copy of Input Waste Table.xlsx", sheet_name='WASTE', header=0).astype({'MEAL CODE': "string", 'INGREDIENT CODE': 'string'})

##### Add unique id column #####

In [350]:
waste_data['id'] = waste_data[['MEAL CODE', 'INGREDIENT CODE']].agg('_'.join, axis=1).astype('string')

#### Product master ####

In [351]:
product_data = pd.read_excel("Input Productmaster.xlsx", sheet_name='Product')

##### Split product data by categorie #####
Store the ids ('hf_nr') in NumPy arrays for easy and fast checking against later.

In [352]:
product_data_ingredient = np.array(product_data[product_data['Categorie'] == 'Ingredient']['Nummer'])
product_data_packaging = np.array(product_data[product_data['Categorie'] == 'Verpakking']['Nummer'])
product_data_HF = np.array(product_data[product_data['Categorie'] == 'Halffabrikaat']['Nummer'])
product_data_gas = np.array(product_data[product_data['Categorie'] == 'Gas']['Nummer'])

### Data cleaning ###

#### BOM ####

##### Split data into recipes #####

In [353]:
recipes = []

for i in range(len(bom_data_raw)):
    # a new recipe starts
    if bom_data_raw[4][i] == 'Omschrijving':
        start_idx = i + 1
        recipe_name = bom_data_raw[4][i + 1]
        recipe_id = bom_data_raw[3][i + 1]

        # the recipe ends
        for j in range(i, len(bom_data_raw)):
            if bom_data_raw[3][j] == 'Kostenaandeel voor dit artikel':
                end_idx = j
                recipe_data = bom_data_raw.iloc[(i + 2):j].drop(range(8, 13), axis='columns').reset_index()
                recipe_data = recipe_data.rename(columns={0: "id_nr", 1: "nr", 2: "Niveau", 3: "hf_nr", 4: "Omschrijving", 5: "Aantal (Basis)", 6: "Basiseenheid", 7: "Materiaalkosten"})
                recipe_data = recipe_data.astype({"id_nr": str, "nr": int, "Niveau": int, "hf_nr": str, "Omschrijving": str, "Aantal (Basis)": float, "Basiseenheid": str, "Materiaalkosten": float})
                recipe_data.insert(loc=2, column="Product Naam", value=[recipe_name for i in range(len(recipe_data))])
                recipes.append(recipe(recipe_name, recipe_id, recipe_data))
                i += j
                break

##### Split recipes into HFs #####

In [354]:
HFs = {}

for recipe in recipes:
    for i in range(len(recipe.data)):
        if recipe.data["hf_nr"][i] in product_data_HF:
            HF_level = recipe.data["Niveau"][i]
            for j in range(i + 1, len(recipe.data)):
                if not recipe.data["Niveau"][j] <= HF_level:
                    j += 1
                else:
                    break
            HFs[f'{recipe.id}_{recipe.data["hf_nr"][i]}'] = HF(name=recipe.data["Omschrijving"][i], id=recipe.data["hf_nr"][i], cost=0.0, weight=recipe.data["Aantal (Basis)"][i], waste=0.0, data=recipe.data.iloc[i + 1 : j + 1 ])

##### Split recipes into ingredients #####
Ingredients are stored in a DataFrame, as they are not complex objects

In [355]:
ingredients = {}

for recipe in recipes:
    for i in range(len(recipe.data)):
        if not (recipe.data["hf_nr"][i] in product_data_HF) and not (recipe.data["hf_nr"][i] in product_data_packaging):
            if not recipe.data["hf_nr"][i] in ingredients.keys():
                ingredients[recipe.data["hf_nr"][i]] = {"Ingredient" : recipe.data["Omschrijving"][i], "id" : recipe.data["hf_nr"][i], "Prijs" : recipe.data["Materiaalkosten"][i], 
                                                        "Eenheid" : recipe.data["Basiseenheid"][i]}

ingredients_df = pd.DataFrame.from_dict(ingredients, orient='index').reset_index(drop=True)

## Cost calculations ##

### Recipes ###

In [356]:
# for recipe in recipes:
#     recipe.cost = recipe.calc_recipe_cost()

### HFs ###

In [357]:
for HF in HFs.values():
    HF.cost = HF.calc_hf_cost()

## BOM redesign ##

Calculate the difficult things (hierarchy dependent).

In [358]:
for recipe in recipes:
    category_column = []
    new_cost_column = [] # nieuwe vvp (new p & old q)
    new_price_column = []
    weight_column = []
    waste_nav_column = []
    waste_fin_column = []
    waste_use_column = []
    
    for i in range(len(recipe.data)):
        item_id = recipe.data['hf_nr'][i]

        # Find updated info
        subsetted_cost_weight_df = cost_weight_data.loc[cost_weight_data['INGREDIENT CODE'] == str(item_id)]
        subsetted_waste_df = waste_data.loc[waste_data['id'] == (recipe.id + '_' + recipe.data['hf_nr'][i])]
        

        # Packaging
        if item_id in product_data_packaging:
            # Category
            category = 'Verpakking'

            # Price, Weight & Cost
            new_price = 0
            weight_kg = 0
            new_cost = 0

            # Waste
            if not len(subsetted_waste_df) == 0:
                waste_nav = subsetted_waste_df['WASTE-NAV'].iloc[0]
                waste_fin = subsetted_waste_df['WASTE-FIN'].iloc[0]
                waste_use = subsetted_waste_df['WASTE-USE'].iloc[0]

            else:
                waste_nav = None
                waste_fin = None
                waste_use = None

        # HFs
        elif item_id in product_data_HF:
            # Category
            category = 'Halffabricaat'

            # Price
            new_price = None

            # Cost
            new_cost = HFs[f'{recipe.id}_{item_id}'].cost
            
            # Weight
            # No new info available
            if len(subsetted_cost_weight_df) == 0:
                weight_kg = None

            else:
                if not subsetted_cost_weight_df['KG'].iloc[0] == None:
                    weight_kg = subsetted_cost_weight_df['KG'].iloc[0] * HFs[f'{recipe.id}_{item_id}'].weight
            
            # Waste
            level = recipe.data['Niveau'][i]

            # HF not level 1
            if level != 1:
                # Loop backwards to find which HF this HF belongs to
                for j in range(i, -1, -1):
                    if recipe.data['Niveau'].iloc[j] == 1:
                        parent_hf_id = recipe.id + '_' + recipe.data['hf_nr'].iloc[j]
                        break
                    
                try:
                    subsetted_waste_special_df = waste_data.loc[waste_data['id'] == parent_hf_id]

                    if not len(subsetted_waste_special_df) == 0:
                        waste_nav = subsetted_waste_special_df['WASTE-NAV'].iloc[0]
                        waste_fin = subsetted_waste_special_df['WASTE-FIN'].iloc[0]
                        waste_use = subsetted_waste_special_df['WASTE-USE'].iloc[0]
                except:
                    print(f'Could not find level 1 HF waste for HF: {item_id}')

                    waste_nav = None
                    waste_fin = None
                    waste_use = None
            
            # HF is level 1
            else:
                if not len(subsetted_waste_df) == 0:
                    waste_nav = subsetted_waste_df['WASTE-NAV'].iloc[0]
                    waste_fin = subsetted_waste_df['WASTE-FIN'].iloc[0]
                    waste_use = subsetted_waste_df['WASTE-USE'].iloc[0]
                
                else:
                    waste_nav = None
                    waste_fin = None
                    waste_use = None

        # Normal ingredient
        elif (item_id in product_data_ingredient) or (item_id in product_data_gas):
            # Category
            if item_id in product_data_ingredient:
                category = 'Ingredient'
            elif item_id in product_data_gas:
                category = 'Gas'
            else:
                print(f'item_id: {item_id} categorie: huh???')
                category = None

            # Price, Weight & Cost
            # No new info available
            if len(subsetted_cost_weight_df) == 0:
                new_price = "Geen nieuwe prijs"
                weight_kg = None
                new_cost = None # TODO - include handling
            
            # New info
            else:
                new_price = subsetted_cost_weight_df["PRICE Q1"].iloc[0]
                new_cost = new_price * recipe.data['Aantal (Basis)'].iloc[i]
                
                if subsetted_cost_weight_df['KG'].iloc[0] == None:
                    weight_kg = recipe.data['Aantal (Basis)'].iloc[i]
                else:
                    weight_kg = subsetted_cost_weight_df['KG'].iloc[0] * recipe.data['Aantal (Basis)'].iloc[i]

            # Waste
            if not len(subsetted_waste_df) == 0:
                waste_nav = subsetted_waste_df['WASTE-NAV'].iloc[0]
                waste_fin = subsetted_waste_df['WASTE-FIN'].iloc[0]
                waste_use = subsetted_waste_df['WASTE-USE'].iloc[0]
            
            else:
                level = recipe.data['Niveau'][i]
                if level != 1:
                    # Loop backwards to find which HF this ingredient belongs to
                    for j in range(i, -1, -1):
                        if recipe.data['Niveau'].iloc[j] == 1:
                            parent_hf_id = recipe.id + '_' + recipe.data['hf_nr'].iloc[j]
                            break
                        
                    try:
                        subsetted_waste_special_df = waste_data.loc[waste_data['id'] == parent_hf_id]

                        if not len(subsetted_waste_special_df) == 0:
                            waste_nav = subsetted_waste_special_df['WASTE-NAV'].iloc[0]
                            waste_fin = subsetted_waste_special_df['WASTE-FIN'].iloc[0]
                            waste_use = subsetted_waste_special_df['WASTE-USE'].iloc[0]
                    except:
                        print(f'Could not find level 1 HF waste for ingredient: {item_id}')

                        waste_nav = None
                        waste_fin = None
                        waste_use = None
        else:
            print(f'item: {item_id} not classified in product master')
            category = None
        
        category_column.append(category)
        new_price_column.append(new_price)
        new_cost_column.append(new_cost)
        weight_column.append(weight_kg)
        waste_nav_column.append(waste_nav)
        waste_fin_column.append(waste_fin)
        waste_use_column.append(waste_use)
    
    recipe.data['Categorie'] = category_column
    recipe.data['Nieuwe prijs Q1'] = new_price_column
    recipe.data['Nieuwe vvp'] = new_cost_column
    recipe.data['Grammage Q1'] = weight_column
    recipe.data['Waste NAV Q1'] = waste_nav_column
    recipe.data['Waste FIN Q1'] = waste_fin_column
    recipe.data['Waste USE Q1'] = waste_use_column

item: 301407G not classified in product master
item: 301407N not classified in product master
item: 301408G not classified in product master
item: 301408N not classified in product master
item: 301409G not classified in product master
item: 301409N not classified in product master
item: 301410G not classified in product master
item: 301410N not classified in product master
item: 301412G not classified in product master
item: 301412N not classified in product master
item: 301413G not classified in product master
item: 301413N not classified in product master
item: 301414G not classified in product master
item: 301414N not classified in product master
item: 301415G not classified in product master
item: 301415N not classified in product master
item: 301416G not classified in product master
item: 301416N not classified in product master
item: 301417G not classified in product master
item: 301417N not classified in product master
item: 301418G not classified in product master
item: 301418N

Calculate the strictly row level stuff.

In [359]:
for recipe in recipes:
    nr_without_waste_col = []
    nr_new_col = []
    costs_new_col = []
    delta_waste_col = []
    delta_price_col = []
    
    for i in range(len(recipe.data)):
        item_id = recipe.data['hf_nr'][i]

        try:
            nr_without_waste = recipe.data['Aantal (Basis)'][i] / (1 + recipe.data['Waste NAV Q1'][i])
        except:
            print(f'for item {item_id} could not calculate nr_without_waste')
            nr_without_waste = None

        try:
            nr_new = nr_without_waste * (1 + recipe.data['Waste USE Q1'][i])
        except:
            print(f'for item {item_id} could not calculate nr_new')
            nr_new = None

        try:
            costs_new = nr_new * recipe.data['Nieuwe prijs Q1'][i]
        except:
            print(f'for item {item_id} could not calculate costs_new')
            costs_new = None

        try:
            delta_waste = (nr_new - recipe.data['Aantal (Basis)'][i]) * recipe.data['Nieuwe prijs Q1'][i]
        except:
            print(f'for item {item_id} could not calculate delta_waste')
            delta_waste = None

        try:
            delta_price = (recipe.data['Nieuwe prijs Q1'][i] - recipe.data['Materiaalkosten'][i]) * nr_new
        except:
            print(f'for item {item_id} could not calculate delta_price')
            delta_price = None
    
        nr_without_waste_col.append(nr_without_waste)
        nr_new_col.append(nr_new)
        costs_new_col.append(costs_new)
        delta_waste_col.append(delta_waste)
        delta_price_col.append(delta_price)

    recipe.data['Aantal (zonder waste)'] = nr_without_waste_col
    recipe.data['Aantal (nieuw)'] = nr_new_col
    recipe.data['Materiaalkosten (nieuw)'] = costs_new_col
    recipe.data['Delta waste'] = delta_waste_col
    recipe.data['Delta prijs'] = delta_price_col

for item 301404 could not calculate nr_without_waste
for item 301404 could not calculate nr_new
for item 301404 could not calculate costs_new
for item 301404 could not calculate delta_waste
for item 301404 could not calculate delta_price
for item 301405 could not calculate nr_without_waste
for item 301405 could not calculate nr_new
for item 301405 could not calculate costs_new
for item 301405 could not calculate delta_waste
for item 301405 could not calculate delta_price
for item 301407G could not calculate nr_without_waste
for item 301407G could not calculate nr_new
for item 301407G could not calculate costs_new
for item 301407G could not calculate delta_waste
for item 301407G could not calculate delta_price
for item 101146 could not calculate nr_without_waste
for item 101146 could not calculate nr_new
for item 101146 could not calculate costs_new
for item 101146 could not calculate delta_waste
for item 101146 could not calculate delta_price
for item 101147 could not calculate nr_with

Fill in the missing cost data for HFs (new p & new q).

In [360]:
for recipe in recipes:
    for i in range(len(recipe.data)):
        item_id = recipe.data['hf_nr'][i]

        if item_id in product_data_HF:
            hf_new_cost = 0.0
            length = len(HFs[f'{recipe.id}_{item_id}'].data)
            for j in range(i + 1, i + length + 1):
                if not recipe.data['Materiaalkosten (nieuw)'][j] == None:
                    if not np.isnan(recipe.data['Materiaalkosten (nieuw)'][j]):
                        hf_new_cost += float(recipe.data['Materiaalkosten (nieuw)'][j])
            recipe.data.at[i, 'Materiaalkosten (nieuw)'] = hf_new_cost

## Output Excel file ##

### Recipes ###

In [361]:
# recipes_dict = {}

# i = 0
# for recipe in recipes:
#     recipes_dict[i] = [recipe.name, recipe.id, recipe.cost]
#     i += 1

# recipes_df = pd.DataFrame.from_dict(recipes_dict, orient='index', columns=['Product', 'id', 'Kostprijs (zonder verpakking) Q1'])

### HFs ###

In [362]:
# HFs_dict = {}

# i = 0
# for HF in HFs:
#     HFs_dict[i] = [HF.name, HF.id, HF.weight, HF.waste, HF.cost]
#     i += 1

# HFs_df = pd.DataFrame.from_dict(HFs, orient='index', columns=['Halffabricaat', 'id', 'Gewicht Q1', 'Waste Q1', 'Kostprijs (zonder verpakking) Q1'])

### BOM ###

In [363]:
frames = []
for recipe in recipes:
    frames.append(recipe.data)

BOM_df = pd.concat(frames)

C:\Users\michi\AppData\Local\Temp\ipykernel_5848\663173341.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  BOM_df = pd.concat(frames)


### Save to Excel ###

In [364]:
with pd.ExcelWriter("Output v2.xlsx") as writer:
    # recipes_df.to_excel(writer, sheet_name="Producten")
    # HFs_df.to_excel(writer, sheet_name="Halffabricaten")
    BOM_df.to_excel(writer, sheet_name="BOM")
    # ingredients_df.to_excel(writer, sheet_name="Ingrediënten")